decoder transformer层的linear1层（dtl1）替换为近似矩阵乘法, 32\*32\*64, 64\*128, 32\*32\*128

In [37]:
import numpy as np
import os
import matmul as mm
import math_util as mu
# import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [38]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [39]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [40]:
linear_name = 'dtl1'
feedback_bits = 256
ncodebooks = 32 # max:64
ncentroids = 16
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [41]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'dx_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'dx_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'dx_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'dx_linear1in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    featurepath_test = 'dx_linear1out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dirs")


weightpath = 'dx_linear1_w_f256.npy'
biaspath = 'dx_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [42]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  PQ
running kmeans in subspace 1/32... mse / {var(X_subs), var(X)}: 0.087, 0.0355
running kmeans in subspace 2/32... mse / {var(X_subs), var(X)}: 0.0473, 0.132
running kmeans in subspace 3/32... mse / {var(X_subs), var(X)}: 0.0602, 0.0621
running kmeans in subspace 4/32... mse / {var(X_subs), var(X)}: 0.112, 0.0591
running kmeans in subspace 5/32... mse / {var(X_subs), var(X)}: 0.138, 0.0934
running kmeans in subspace 6/32... mse / {var(X_subs), var(X)}: 0.143, 0.0541
running kmeans in subspace 7/32... mse / {var(X_subs), var(X)}: 0.168, 0.0746
running kmeans in subspace 8/32... mse / {var(X_subs), var(X)}: 0.0746, 0.084
running kmeans in subspace 9/32... mse / {var(X_subs), var(X)}: 0.0946, 0.0382
running kmeans in subspace 10/32... mse / {var(X_subs), var(X)}: 0.0783, 0.0306
running kmeans in subspace 11/32... mse / {var(X_subs), var(X)}: 0.0677, 0.0382
running kmeans in subspace 12/32... mse / {var(X_subs), var(X)}: 0.067, 0.0487
running kmeans in subspace 13/32... m

In [43]:
# type(est3)

In [44]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [45]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 0.60410035  0.17575683  0.21739253 ...  0.15072177 -0.64169884
   1.0388054 ]
 [ 0.32988656 -0.16061205 -0.47362614 ... -0.22586511 -0.08961515
   1.2728012 ]
 [-0.54394126 -0.05823891 -0.2798484  ...  0.10684757  0.04931981
  -0.7307874 ]
 ...
 [ 1.1232784   0.24522431  1.2338115  ...  0.04103627 -0.20295683
   0.7974973 ]
 [ 0.878314    0.24156813  0.52451193 ...  0.06297337 -0.4515773
   1.1594594 ]
 [ 1.0501547   0.28544232  1.2703733  ...  0.08491047 -0.1883321
   0.812122  ]]
y_out_last.shape:  (1024000, 128)
y_out_last_re.shape:  (1000, 32, 32, 128)
